# Transforming cost functions using MinimallyDisruptiveCurves.jl

The premise of MinimallyDisruptiveCurves.jl is to
> move as far away as possible from the initial parameters, while keeping model behaviour as similar as possible.

(in other words, while keeping a cost function as low as possible).

The words *as far away as possible* imply some metric on the space of parameters. A lot of key workflows involve manipulating this metric. For instance, by

- fixing/freeing parameters that the minimally disruptive curve can change
- looking at relative, rather than absolute changes in parameters
- biasing parameters to have a larger/smaller influence on the metric, so that minimally disruptive curves are encouraged (not /) to align with them
- something custom.

Each of these corresponds to a **transformation** of parameter space. The easiest way to do all of these things is by **reparameterising** the cost function $C(\theta)$. We take a transformation of parameter space: $T(\theta)$, and a new cost function $D$ satisfying
$$ D[T(\theta)] = C(\theta). $$

**The purpose of this notebook is to show you an easy way to make/perform these reparameterisations.**

The (slight) complication is that cost functions compatible with MinimallyDisruptiveCurves.jl must be **differentiable**. That is, they have two methods:
```
## method 1
function cost(𝜃)
    ...
    return cost
end

## method 2
function cost(𝜃, grad_template)
    ....
    grad_template[:] = ∇C    # mutate to get gradient wrt parameters
    
    return cost
end

```

So we want an easy way of applying composable transformations to cost functions, which also recompute the gradient. Our solution 